In [ ]:
import logging
 
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Log等级总开关
log_file_abs = "flask.log"
 
stream_handler = logging.StreamHandler()  # 日志控制台输出
 
handler = logging.FileHandler(log_file_abs, mode='w', encoding='UTF-8')  # 日志文件输出
handler.setLevel(logging.DEBUG)
 
# 控制台输出格式
stream_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
# 文件输出格式
logging_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
handler.setFormatter(logging_format)  # 为改处理器handler 选择一个格式化器
stream_handler.setFormatter(stream_format)
 
logger.addHandler(handler)  # 为记录器添加 处理方式Handler
logger.addHandler(stream_handler)
 
logger.info("------logger.info-----")

In [ ]:
import time
print(time.localtime())

In [ ]:
print("{:.6f}".format(1))

In [ ]:
def function1():
    print("function1")

def function2():
    print("function2")

def function3():
    print("function3")

def call_fun_by_str(fun_str):
    eval(fun_str)()

call_fun_by_str("function2")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

x = torch.rand(2,64,128,128)

# y = F.interpolate(x,size=(128,128), scale_factor=None, mode='bilinear', align_corners=None, recompute_scale_factor=None)
# print(y.shape)
    
trans_conv = nn.ConvTranspose2d(in_channels=64, out_channels=16,kernel_size=2, stride=2)
y = trans_conv(x)
print(y.shape)

# pixel = nn.PixelShuffle(4)
# y = pixel(x)
# print(y.shape)

In [ ]:
import torch
import segmentation_models_pytorch as smp
model1 = smp.DeepLabV3Plus(
        encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset)
    )

model2 = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)
print(model1)

In [ ]:
import torch
a = torch.nn.Conv2d(3, 256, kernel_size=7, stride=4,
                              padding=(7 // 2, 7 // 2))
x = torch.rand(1,3,512,512)
y = a(x)
print(y.shape)

In [1]:
import torch
from thop import profile
from model import transunet_m,swinunet_m,deeplabv3p_smp,unet_smp,segformer_m,pspnet_smp,segnet_m
model = segformer_m()
x = torch.randn(2, 3, 512, 512)
flops , params = profile(model,inputs=(x,))
print(flops)
print(params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
6812336128.0
3933346.0


In [1]:
import timm
import torch
model_resnet = timm.list_models('*unet*')
print(model_resnet)

# # #mobilenetv3_large_075
# net = timm.create_model('resnet101', pretrained=False,num_classes=0,global_pool='')
# print(net)

[]


In [9]:
import torch
from timm.models.efficientnet_blocks import DepthwiseSeparableConv

x = torch.rand(2,256,256,256)
net = DepthwiseSeparableConv(256,128,3)
print(net)
print(net(x).shape)


DepthwiseSeparableConv(
  (conv_dw): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=256, bias=False)
  (bn1): BatchNormAct2d(
    256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): ReLU(inplace=True)
  )
  (se): Identity()
  (conv_pw): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn2): BatchNormAct2d(
    128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): Identity()
  )
  (drop_path): Identity()
)
torch.Size([2, 128, 256, 256])


In [2]:
import torch,onnx
from model.swin_unet import swinunet_m

net = swinunet_m(
        num_classes=2
    )
model_name = 'net.onnx'#保存ONNX的文件名字
dummy_input = torch.randn(1, 3, 512, 512)
torch.onnx.export(net, dummy_input, model_name, input_names=['input'], output_names=['output'])

d:\31890\Desktop\codefile\mseg\model\swin_unet.py:171: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert L == H * W, "input feature has wrong size"
d:\31890\Desktop\codefile\mseg\model\swin_unet.py:32: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  B = int(windows.shape[0] / (H * W / window_size / window_size))
d:\31890\Desktop\codefile\mseg\model\swin_unet.py:306: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

